In [1]:
import sys
import os

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp
import time
#disable jax JIT
#jax.config.update("jax_disable_jit", True)

from tqdm.autonotebook import trange
from functools import partial
from JAxtar.bgpq import HashTableHeapValue, BGPQ

/tmp/ipykernel_808417/3628738232.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [3]:
max_size = 10
group_size = 2

In [4]:
heap = BGPQ.make_heap(max_size, group_size, HashTableHeapValue)
insert = jax.jit(heap.insert)
delete_mins = jax.jit(heap.delete_mins)

2024-07-29 09:31:12.841104: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.5.82). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [5]:
heap_key = jax.random.uniform(jax.random.PRNGKey(0), shape=(3,), minval=0, maxval=10)
value = jax.vmap(HashTableHeapValue.default)(jnp.arange(3))
make_batch = BGPQ.make_batched(heap_key, value, group_size)

In [6]:
for i in range(10):
    heap_key = jax.random.uniform(jax.random.PRNGKey(i), shape=(1,), minval=0, maxval=10)
    value = jax.vmap(HashTableHeapValue.default)(jnp.arange(1))
    block_key, block_val = BGPQ.make_batched(heap_key, value, group_size)
    heap, buffer_oveflow = insert(heap, block_key[0], block_val[0])
    for i in range(10):
        print(heap.key_store[i])
    print(heap.size)
    print(buffer_oveflow)

Traced<ShapedArray(int32[5])>with<DynamicJaxprTrace(level=2/0)>
[4.1845703       inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[0]
False
[1.1815023 4.1845703]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[0]
False
[1.1815023 4.1845703]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[0]
False
[1.1815023 4.1845703]
[4.2402163 8.653928 ]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[1]
True
[1.1815023 1.6215026]
[4.2402163 8.653928 ]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[1]
False
[1.1815023 1.6215026]
[4.1845703 4.2402163]
[6.3434052 8.653928 ]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[2]
True
[1.1815023 1.6215026]
[4.1845703 4.2402163]
[6.3434052 8.653928 ]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[2]
False
[1.1815023 1

In [7]:
size = int(heap.size[0]+1)
for i in range(size):
    heap, min_key, min_val = delete_mins(heap)

    print("----------------")
    print(min_key)
    print("----------------")
    for i in range(10):
        print(heap.key_store[i])
    print(heap.size)

----------------
[1.1815023 1.6215026]
----------------
[5.1601324 6.5711427]
[4.1845703 4.2402163]
[4.1845703 4.2402163]
[inf inf]
[5.1601324 6.5711427]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[3]
----------------
[5.1601324 6.5711427]
----------------
[5.1601324 6.5711427]
[5.1601324 6.5711427]
[inf inf]
[inf inf]
[5.1601324 6.5711427]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[2]
----------------
[5.1601324 6.5711427]
----------------
[5.1601324 6.5711427]
[inf inf]
[6.5711427 6.5711427]
[6.5711427 6.5711427]
[6.5711427 6.5711427]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[1]
----------------
[5.1601324 6.5711427]
----------------
[inf inf]
[inf inf]
[6.5711427 6.5711427]
[6.5711427 6.5711427]
[6.5711427 6.5711427]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[0]
----------------
[inf inf]
----------------
[inf inf]
[inf inf]
[6.5711427 6.5711427]
[6.5711427 6.5711427]
[6.5711427 6.5711427]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[inf inf]
[4294967295]